## Introduction to Building AI Agents
In this notebook, we embark on a journey to build an AI agent using Langchain. This comprehensive guide synthesizes the knowledge acquired from previous lessons, providing a step-by-step approach to constructing an agent capable of performing tasks autonomously.

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

#### Setting Up Custom Tools for Agents
To enhance our agent's capabilities, we will integrate custom tools defined in the previous notebook. These tools, including a URL content fetcher and a summarizer, will empower our agent to retrieve and process information efficiently.

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.tools import tool

@tool
def fetch_url_content(url: str) -> str:
    """Fetches the content of a specified URL. This tool is essential for agents requiring access to web-based information."""
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_anthropic import ChatAnthropic

@tool
def summarize_content(content: str) -> str:
    """Generates a concise summary of text-based content. Optimized for summarization tasks, this tool aids in distilling essential information."""
    system_message = "Please provide a concise summary for the content"
    human_message = "Content: " + str(content)
    
    messages = [
        SystemMessage(content=system_message),
        HumanMessage(content=human_message),
    ]
    model = ChatAnthropic(model="claude-3-haiku-20240229")
    parser = StrOutputParser()
    
    chain = model | parser
    summary = chain.invoke(messages)
    return summary

In [4]:
# Integrating custom tools with the agent
tools = [fetch_url_content, summarize_content]

In [5]:
# Selecting the LLM model for the agent's core
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o")

#### Building the Agent with Langgraph
We will utilize Langgraph's ReAct agent framework to construct our AI agent. This modern approach simplifies the development process, allowing for the seamless integration of tools and models to create a versatile and autonomous agent.

In [6]:
from langgraph.prebuilt import create_react_agent

# Creating the agent executor with the selected model and tools
agent_executor = create_react_agent(model, tools)

In [7]:
# Testing the agent's initial response
agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

{'messages': [HumanMessage(content='hi!', id='d18574e4-eb17-4a4c-97c6-242fbfc4f410'),
  AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 86, 'total_tokens': 96}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-4fcf143c-c288-4266-a18f-f31a775fb96f-0', usage_metadata={'input_tokens': 86, 'output_tokens': 10, 'total_tokens': 96})]}

In [ ]:
# Requesting the agent to summarize web content
# Note: The execution time may vary based on the content size due to the LLM's token processing limit.
response = agent_executor.invoke({"messages": [HumanMessage(content="can you please summarize the content of this url https://lilianweng.github.io/posts/2023-06-23-agent/")]})
response["messages"]